<a href="https://colab.research.google.com/github/ShubhDixit20/Sagemaker_test/blob/main/Sagemaker_test_refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd Cow Neck AI

/content/drive/MyDrive/Cow Neck AI


In [ ]:
!ls

'Copy of Cow1.gsheet'   HelperFunction.py    Untitled0.ipynb
'Copy of Cow.csv'       PreProcessed.csv     v1.ipynb
 Cow.csv	        Sagemaker_test.csv


In [ ]:
import threading
import atexit
import os
import shutil
import pandas as pd
import os
import copy
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib
import warnings
from math import *

In [ ]:
WINDOW_SIZE = 1
SAMPLING_RATE = 25
STEP_SIZE = 1
WINDOW_LENGTH = int(int(SAMPLING_RATE) * float(WINDOW_SIZE))
STEP_LENGTH = int(round(int(SAMPLING_RATE) * float(STEP_SIZE)))
data_temp = pd.DataFrame()
data_combined = pd.DataFrame()

In [17]:
# Here we need to pass the path of the raw data csv file.
PATH_FILE = r'Copy of cow5.csv'

In [18]:
df = pd.read_csv(PATH_FILE)
df

,TimeStamp_UNIX,TimeStamp_JST,AccX,AccY,AccZ,Label
0,1591930195711,11:49:55.711000,-0.110,0.044,-1.041,NaN
1,1591930195751,11:49:55.751000,-0.110,0.045,-1.040,NaN
2,1591930195791,11:49:55.791000,-0.110,0.045,-1.039,NaN
3,1591930195831,11:49:55.831000,-0.110,0.044,-1.040,NaN
4,1591930195871,11:49:55.871000,-0.110,0.044,-1.040,NaN
...,...,...,...,...,...,...
134899,1591935604838,13:20:04.838000,-0.112,0.555,0.829,ETC
134900,1591935604878,13:20:04.878000,-0.114,0.552,0.824,ETC
134901,1591935604918,13:20:04.918000,-0.115,0.555,0.820,ETC
134902,1591935604958,13:20:04.958000,-0.116,0.558,0.817,ETC


In [21]:
df = df.dropna(axis=0, subset=['Label'])
df.reset_index(drop=True, inplace=True)
df

,TimeStamp_UNIX,TimeStamp_JST,AccX,AccY,AccZ,Label
0,1591931139067,12:05:39.067000,-0.212,0.432,0.967,ETC
1,1591931139107,12:05:39.107000,-0.208,0.415,0.987,ETC
2,1591931139147,12:05:39.147000,-0.159,0.387,0.955,ETC
3,1591931139187,12:05:39.187000,-0.119,0.381,0.946,ETC
4,1591931139227,12:05:39.227000,-0.109,0.368,0.930,ETC
...,...,...,...,...,...,...
73355,1591935604838,13:20:04.838000,-0.112,0.555,0.829,ETC
73356,1591935604878,13:20:04.878000,-0.114,0.552,0.824,ETC
73357,1591935604918,13:20:04.918000,-0.115,0.555,0.820,ETC
73358,1591935604958,13:20:04.958000,-0.116,0.558,0.817,ETC


In [ ]:
data_formatted = df.copy()
# data_formatted = data_formatted.drop(['TimeStamp_JST'], axis=1)
data_formatted = data_formatted.drop(['Label'], axis=1)
data_formatted

,AccX,AccY,AccZ
0,-0.455,0.630,0.642
1,-0.458,0.631,0.637
2,-0.459,0.639,0.633
3,-0.456,0.646,0.634
4,-0.450,0.649,0.632
...,...,...,...
181467,0.040,1.062,0.054
181468,0.028,1.078,-0.021
181469,0.100,1.152,-0.037
181470,0.189,1.228,-0.007


In [29]:
data_unix = df[['TimeStamp_UNIX', 'Label']]
data_unix

,TimeStamp_UNIX,Label
0,1591931139067,ETC
1,1591931139107,ETC
2,1591931139147,ETC
3,1591931139187,ETC
4,1591931139227,ETC
...,...,...
73355,1591935604838,ETC
73356,1591935604878,ETC
73357,1591935604918,ETC
73358,1591935604958,ETC


In [28]:
data_formatted.to_csv(r'PreProcessed.csv', index=False)
data_formatted

,AccX,AccY,AccZ
0,-0.212,0.432,0.967
1,-0.208,0.415,0.987
2,-0.159,0.387,0.955
3,-0.119,0.381,0.946
4,-0.109,0.368,0.930
...,...,...,...
73355,-0.112,0.555,0.829
73356,-0.114,0.552,0.824
73357,-0.115,0.555,0.820
73358,-0.116,0.558,0.817


In [33]:
tf = pd.read_csv(r'PreProcessed.csv')
test_object = Path(r'PreProcessed.csv').stem
labels = data_unix['Label']
labels

0        ETC
1        ETC
2        ETC
3        ETC
4        ETC
        ... 
73355    ETC
73356    ETC
73357    ETC
73358    ETC
73359    ETC
Name: Label, Length: 73360, dtype: object

In [34]:
range_end = len(data_formatted) - WINDOW_LENGTH
range_end

73335

In [ ]:
temp_df = data_formatted[['AccX', 'AccY', 'AccZ']]
temp_df

,AccX,AccY,AccZ
0,-0.455,0.630,0.642
1,-0.458,0.631,0.637
2,-0.459,0.639,0.633
3,-0.456,0.646,0.634
4,-0.450,0.649,0.632
...,...,...,...
181467,0.040,1.062,0.054
181468,0.028,1.078,-0.021
181469,0.100,1.152,-0.037
181470,0.189,1.228,-0.007


In [ ]:
data_selected = temp_df[0:WINDOW_LENGTH]
data_selected

,AccX,AccY,AccZ
0,-0.455,0.630,0.642
1,-0.458,0.631,0.637
2,-0.459,0.639,0.633
3,-0.456,0.646,0.634
4,-0.450,0.649,0.632
...,...,...,...
495,-0.441,0.621,0.671
496,-0.442,0.622,0.669
497,-0.439,0.626,0.663
498,-0.438,0.629,0.664


In [ ]:
# test_object = Path(PATH_FILE).stem
# data = pd.DataFrame(pd.read_csv(PATH_FILE))
# labels = data['Label']
# data_formatted = data.drop(['Label'], axis=1).copy()
# range_end = len(data_formatted) - WINDOW_LENGTH
# data_temp = pd.DataFrame()
# data_formatted

In [ ]:
for i in range(0, range_end, STEP_LENGTH):
    data_selected = data_formatted[i:i+WINDOW_LENGTH]
    req_timestamp = data_unix['TimeStamp_UNIX'][i:i+WINDOW_LENGTH]
    req_label = data_unix['Label'][i:i+WINDOW_LENGTH]

    #Features to be extracted.
    data_kurt = data_selected.kurtosis()
    #data_mad = data_selected.mad()
    data_mean = data_selected.mean()
    data_median = data_selected.median()
    data_min = data_selected.min()
    data_max = data_selected.max()
    data_quan_25 = data_selected.quantile(q=0.25)
    data_quan_50 = data_selected.quantile(q=0.5)
    data_quan_75 = data_selected.quantile(q=0.75)
    data_skew = data_selected.skew()
    data_sum = data_selected.sum()
    data_std = data_selected.std()
    data_var = data_selected.var()

    SignalMagnitudeArea = abs(data_formatted.loc[i][0]) + abs(data_formatted.loc[i][1]) + abs(data_formatted.loc[i][2])

    VectorMagnitude = sqrt(abs(data_formatted.loc[i][0] + data_formatted.loc[i][1] + data_formatted.loc[i][2]))

    MovementVariation = abs(data_formatted.loc[i][0]-data_formatted.loc[i+1][0]) + abs(data_formatted.loc[i][1]-data_formatted.loc[i+1][1]) + abs(data_formatted.loc[i][2]-data_formatted.loc[i+1][2])

    Energy = ((data_formatted.loc[i][0])**2 + (data_formatted.loc[i][1])**2 + (data_formatted.loc[i][2])**2)**2

    Entropy = (1+(data_formatted.loc[i][0] + data_formatted.loc[i][1] + data_formatted.loc[i][2]))**2 * log(abs(1+data_formatted.loc[i][0] + data_formatted.loc[i][1] + (data_formatted.loc[i][2])**2))

    data_combined_hor = pd.concat(
                [pd.Series(test_object), pd.Series(labels.loc[i]), pd.Series(data_formatted.loc[i]),
                 data_kurt, data_mean, data_median, data_min, data_max, data_quan_25,
                 data_quan_50, data_quan_75, data_skew, data_sum, data_std, data_var,
                 pd.Series(SignalMagnitudeArea), pd.Series(VectorMagnitude), pd.Series(MovementVariation), pd.Series(Energy), pd.Series(Entropy)], axis=0, ignore_index=True)

    data_temp = pd.concat([data_temp, data_combined_hor], axis=1, ignore_index=True)
data_combined = pd.concat([data_combined, data_temp], ignore_index = True)

print('data: ',
                data_formatted.loc[i][0],
                data_formatted.loc[i][1],
                data_formatted.loc[i][2])

data_combined.to_csv('PreProcessed.csv', sep=',', index=False)


#     data_temp = pd.concat([data_temp, data_combined_hor], axis=0, ignore_index=True)
# data_temp.to_csv('Sagemaker_test.csv', sep=',', index = False)

# data_combined = pd.concat([data_combined, data_temp], ignore_index=True)
# # data_combined.to_csv('Sagemaker_test.csv', sep=',', index = False)

# print(len(data_temp), len(data_combined), len(data_combined_hor))

# data_combined.columns = ['test_object', 'label', 'AccX', 'AccY', 'AccZ', 'AccX_kurt', 'AccY_kurt', 'AccZ_kurt',
#                         'AccX_mean', 'AccY_mean', 'AccZ_mean', 'AccX_median', 'AccY_median',
#                         'AccZ_median', 'AccX_min', 'AccY_min', 'AccZ_min', 'AccX_max', 'AccY_max', 'AccZ_max',
#                         'AccX_quan_25', 'AccY_quan_25', 'AccZ_quan_25', 'AccX_quan_50', 'AccY_quan_50', 'AccZ_quan_50',
#                         'AccX_quan_75', 'AccY_quan_75', 'AccZ_quan_75', 'AccX_skew', 'AccY_skew', 'AccZ_skew', 'AccX_sum',
#                         'AccY_sum', 'AccZ_sum', 'AccX_std', 'AccY_std', 'AccZ_std', 'AccX_var', 'AccY_var', 'AccZ_var',
#                         'SignalMagnitudeArea', 'VectorMagnitude', 'MovementVariation', 'Energy', 'Entropy']

data:  0.117 1.046 -0.142
